In [78]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() 
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import re

In [79]:
# import data set
df_housing = pd.read_excel("Land Value Sales 2020.xlsx")
df_housing.head()

,Unnamed: 0,list-card-link href,list-card-addr,ADDRESS,city,state,Metro,zip,price,sq ft,...,pricesqft,list-card-label,list-card-label 2,list-card-variable-text,walkscore,description,LATITUDE,LONGITUDE,list-card-link src,list-card-save-content
0,"{""@type"":""SingleFamilyResidence"",""@context"":""h...",https://www.zillow.com/homedetails/0-Bayfield-...,"0 Bayfield Ave, Arverne, NY 11692",Bayfield Ave,Arverne,NY,New York,11692.0,360000.0,8481.0,...,42.447825,sqft lot,NaN,Sold 08/06/2018,44,Car-Dependent,40.599041,-73.799969,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",NaN
1,"{""@type"":""SingleFamilyResidence"",""@context"":""h...",https://www.zillow.com/homedetails/3045-23rd-A...,"3045 23rd, Astoria, NY 11102",3045 23rd,Astoria,NY,New York,11102.0,825000.0,2000.0,...,412.5,sqft lot,NaN,Sold 10/29/2020,97,Walker's Paradise,40.768025,-73.927269,https://maps.googleapis.com/maps/api/streetvie...,NaN
2,"{""@type"":""SingleFamilyResidence"",""@context"":""h...",https://www.zillow.com/homedetails/18-11-21-Dr...,"18-11 21 Drive, Astoria, NY 11105",18-11 21 Drive,Astoria,NY,New York,11105.0,3000000.0,3025.0,...,991.735537,sqft lot,NaN,Sold 01/16/2019,75,Very Walkable,40.782982,-73.918476,https://maps.googleapis.com/maps/api/streetvie...,NaN
3,"{""@type"":""SingleFamilyResidence"",""@context"":""h...",https://www.zillow.com/homedetails/18-15-21-Dr...,"18-15 21 Drive, Astoria, NY 11105",18-15 21 Drive,Astoria,NY,New York,11105.0,3000000.0,2888.0,...,1038.781163,sqft lot,NaN,Sold 01/16/2019,75,Very Walkable,40.782897,-73.918410,https://maps.googleapis.com/maps/api/streetvie...,NaN
4,"{""@type"":""SingleFamilyResidence"",""@context"":""h...",https://www.zillow.com/homedetails/10297-E-Mis...,"10297 E Mississippi Ave, Aurora, CO 80247",10297 E Mississippi Ave,Aurora,CO,Denver,80247.0,300000.0,40510.8,...,7.405433,acres lot,NaN,Sold 10/08/2020,61,Somewhat Walkable,39.697108,-104.869217,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",NaN


# EDA and data prep

In [80]:
# Remove external links as they will not be used in evaluation
del df_housing["list-card-link href"]
del df_housing["list-card-link src"]

In [81]:
# remove column list-card-addr 
del df_housing["list-card-addr"]


In [82]:
# Delete unnamed column as all residences are single family residences
del df_housing["Unnamed: 0"]

In [83]:
df_housing.head()

,ADDRESS,city,state,Metro,zip,price,sq ft,acreage,pricesqft,list-card-label,list-card-label 2,list-card-variable-text,walkscore,description,LATITUDE,LONGITUDE,list-card-save-content
0,Bayfield Ave,Arverne,NY,New York,11692.0,360000.0,8481.0,0.194697,42.447825,sqft lot,NaN,Sold 08/06/2018,44,Car-Dependent,40.599041,-73.799969,NaN
1,3045 23rd,Astoria,NY,New York,11102.0,825000.0,2000.0,0.045914,412.5,sqft lot,NaN,Sold 10/29/2020,97,Walker's Paradise,40.768025,-73.927269,NaN
2,18-11 21 Drive,Astoria,NY,New York,11105.0,3000000.0,3025.0,0.069444,991.735537,sqft lot,NaN,Sold 01/16/2019,75,Very Walkable,40.782982,-73.918476,NaN
3,18-15 21 Drive,Astoria,NY,New York,11105.0,3000000.0,2888.0,0.066299,1038.781163,sqft lot,NaN,Sold 01/16/2019,75,Very Walkable,40.782897,-73.918410,NaN
4,10297 E Mississippi Ave,Aurora,CO,Denver,80247.0,300000.0,40510.8,0.930000,7.405433,acres lot,NaN,Sold 10/08/2020,61,Somewhat Walkable,39.697108,-104.869217,NaN


In [84]:
# Get info about data 
df_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3625 entries, 0 to 3624
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ADDRESS                  3625 non-null   object 
 1   city                     3625 non-null   object 
 2   state                    3625 non-null   object 
 3   Metro                    3625 non-null   object 
 4   zip                      3617 non-null   float64
 5   price                    3625 non-null   float64
 6   sq ft                    3552 non-null   float64
 7   acreage                  3625 non-null   float64
 8   pricesqft                3625 non-null   object 
 9   list-card-label          3625 non-null   object 
 10  list-card-label 2        1724 non-null   object 
 11  list-card-variable-text  3625 non-null   object 
 12  walkscore                3625 non-null   int64  
 13  description              3625 non-null   object 
 14  LATITUDE                

In [85]:
# Count the number of null values each column has  
print(df_housing.isnull().sum())

ADDRESS                       0
city                          0
state                         0
Metro                         0
zip                           8
price                         0
sq ft                        73
acreage                       0
pricesqft                     0
list-card-label               0
list-card-label 2          1901
list-card-variable-text       0
walkscore                     0
description                   0
LATITUDE                      0
LONGITUDE                     0
list-card-save-content     2856
dtype: int64


In [86]:
df_housing.shape

(3625, 17)

Since majority of the columns "list-card-save-content" is null and is only saved for Chicago homes, we can delete that column.

In [87]:
del df_housing["list-card-save-content"]

List card label 2 is a label that tells if the housing property was sold. However, we already have a column that says when the property was sold in "list-card-variable-text." We can del this column too

In [88]:
del df_housing["list-card-label 2"]

There are some null values in sq ft which isn't that bad of a fix. We can convert acerage to sq ft. (1 acre = 43560 sq ft)

In [89]:
df_housing.loc[df_housing["sq ft"].isna(), "sq ft"] = df_housing["acreage"] * 43560

Zip code

In [90]:
df_housing.loc[df_housing["zip"].isna(), "zip"]

210    NaN
417    NaN
1273   NaN
1274   NaN
1456   NaN
1984   NaN
1989   NaN
3095   NaN
Name: zip, dtype: float64

Check if states are in the correct metro area

New York = NY (NJ acceptable)
Denver = CO
LA = CA
Seattle = WA
Chicago = IL
San Francisco = CA
Philadelphia = PA

In [91]:
df_housing["Metro"].unique()

array(['New York', 'Denver', 'Los Angeles', 'Seattle', 'Chicago',
       'Philadelphia', 'San Francisco'], dtype=object)

In [96]:
df_housing["state"].unique()

array(['NY', 'CO', 'CA', 'WA', 'AZ', 'IL', 'PA', 'MA', 'MD'], dtype=object)

In [107]:
# Delete Arizona, Mass, Maryland properties
df_housing = df_housing.drop(df_housing[df_housing['state'] == "AZ"].index)
df_housing = df_housing.drop(df_housing[df_housing['state'] == "MA"].index)
df_housing = df_housing.drop(df_housing[df_housing['state'] == "MD"].index)

In [108]:
df_housing["state"].unique()

array(['NY', 'CO', 'CA', 'WA', 'IL', 'PA'], dtype=object)

In [109]:
# Count the number of null values each column has  
print(df_housing.isnull().sum())

ADDRESS                    0
city                       0
state                      0
Metro                      0
zip                        7
price                      0
sq ft                      0
acreage                    0
pricesqft                  0
list-card-label            0
list-card-variable-text    0
walkscore                  0
description                0
LATITUDE                   0
LONGITUDE                  0
dtype: int64
